# Markdown

- 1- Authentication,
- 2- Get CSV file.
- 3- Get unique tickers in order for us to search tweets that relate to them on twitter.
- 4- Download the tweets that have those tickers. <------- Here.
- 5- Create a model that can classify those tweets into Positive/Negative.
- 6- Use the model in step 5 to classify the downloaded tweets in step 4.
- 7- get only the positive tweets.
- 8- Analysis.

In [9]:
# Import Libraries
import pandas as pd 
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt #to display our wordcloud
import pandas as pd
import numpy as np #to get the color of our image
import os
import nltk
import pycountry
import re
import string
from dotenv import load_dotenv
import requests
import json
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image #to load out image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\omarm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Helper

In [22]:
# Setting Up, basic Sentiment Analysis review
def percentage(part,whole):
    return 100 * float(part)/float(whole) 


# Valid value.
def getValidValue():
    """
    return: float
    """
    i = 1
    while (type(i) != float):
        i = input("Enter the amounts of tweet to extract per ticker:  ")
        try:
            i = float(i)
            if type(i) == float:
                break
        except:
            print("Please enter a valid numbers.... Thank you.")
    return i

# Authentication.

In [10]:
consumerKey = "lh8PpKHuWwF8h6DiTnL6GCSns"
consumerSecret = "bR2tt7BitEqstnOGE8Vjm88eFkOk0frTyd72AR76I2RWAymgY1"
accessToken = "243688593-4QY2mdaSjgz4qDNwFWhrBsCW6ZBJLkBWHkBxCoo4"
accessTokenSecret = "2mF3OGsdZGP5sYZSC4fqihyp0bWFkT2o0T0rNSFH6Crm1"

In [12]:
# Authentication
# consumerKey = os.getenv("TWITTER_API_KEY")
# consumerSecret = os.getenv("TWITTER_SECRET_KEY")
# accessToken = os.getenv("TWITTER_ACCESS_TOKEN")
# accessTokenSecret = os.getenv("TWITTER_ACCESS_TOKEN_SECRET")

auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [13]:
df = pd.read_csv("https://raw.githubusercontent.com/joebary/Project_3_Team_1/main/ESG_data.csv")

In [15]:
df.head(3)

,Date,Total-Score,E-Score,S-Score,G-Score,Ticker
0,2014-09-01,61.0,74.0,45.0,62.0,AAPL
1,2014-10-01,60.0,74.0,45.0,62.0,AAPL
2,2014-11-01,61.0,74.0,45.0,62.0,AAPL


In [20]:
keywords = list(df["Ticker"].unique())
print(keywords)
len(keywords)

['AAPL', 'MSFT', 'AMZN', 'FB', 'TSLA', 'GOOGL', 'JNJ', 'JPM', 'V', 'UNH', 'PG', 'NVDA', 'DIS', 'MA', 'HD', 'PYPL', 'BAC', 'VZ', 'CMCSA', 'ADBE', 'NFLX', 'INTC', 'T', 'MRK', 'PFE', 'WMT', 'CRM', 'TMO', 'ABT', 'PEP', 'KO', 'XOM', 'CSCO', 'ABBV', 'NKE', 'AVGO', 'QCOM', 'CVX', 'ACN', 'COST', 'MDT', 'MCD', 'NEE', 'TXN', 'DHR', 'HON', 'UNP', 'BMY', 'WFC', 'C', 'AMGN', 'LLY', 'PM', 'SBUX', 'LOW', 'ORCL', 'IBM', 'UPS', 'BA', 'MS', 'BLK', 'RTX', 'CAT', 'GS', 'NOW', 'GE', 'MMM', 'INTU', 'CVS', 'AMT', 'TGT', 'ISRG', 'DE', 'CHTR', 'BKNG', 'SCHW', 'MU', 'AMAT', 'LMT', 'FIS', 'TJX', 'ANTM', 'MDLZ', 'SYK', 'CI', 'ZTS', 'AXP', 'SPGI', 'GILD', 'TMUS', 'MO', 'LRCX', 'BDX', 'ADP', 'CSX', 'CME', 'PLD', 'CB', 'CL', 'TFC', 'ADSK', 'ATVI', 'USB', 'PNC', 'DUK', 'FISV', 'CCI', 'ICE', 'SO', 'NSC', 'APD', 'GPN', 'VRTX', 'EQIX', 'ITW', 'SHW', 'D', 'FDX', 'DD', 'HUM', 'EL', 'ADI', 'MMC', 'ECL', 'ILMN', 'EW', 'PGR', 'GM', 'DG', 'BSX', 'NEM', 'ETN', 'COF', 'REGN', 'EMR', 'COP', 'AON', 'WM', 'HCA', 'MCO', 'NOC', 'FCX

307

# Fetching Tweets by Hashtag or Keyword

In [ ]:
noOfTweets = 1
noOfTweets = getValidValue()

# counts
positive  = 0
negative = 0
neutral = 0

# score
polarity = 0


tweets = []
neutral_list = []
negative_list = []
positive_list = []

#temp_list = tweepy.Cursor(api.search_tweets, q='AAPL').items(noOfTweet)

In [52]:
# Dates
date_since = "2021-10-01"
date_until = "2021-12-01"

for keyword in keywords:
    tweets.append(tweepy.Cursor(api.search_tweets, q = keyword ,lang="en", since=date_since, until=date_until).items(noOfTweets))


tweepy.cursor.ItemIterator

In [ ]:
#create an empty list to store results
#tweets_list=[]

# iterate throw the results of the api search
#for item in tweets:
    #create a dict object with the values that we want to store it
    #myTweet = { "tweet_id": item.id_str,
    #            "location": item.user.location, 
    #            "text": item.text, 
    #            "tweet_date": item.created_at
    #          }

# This is going to analyze the text

In [ ]:
for tweet in tweets:
    
    tweet_list.append(tweet.text)
    analysis = TextBlob(tweet.text)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    polarity += analysis.sentiment.polarity
    
    if neg > pos:
        negative_list.append(tweet.text)
        negative += 1

    elif pos > neg:
        positive_list.append(tweet.text)
        positive += 1
    
    elif pos == neg:
        neutral_list.append(tweet.text)
        neutral += 1

positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')


In [ ]:
# Dataframe, Save as CSV, EXCEL -----> GITHUB/AWS